# Momentum ETF Analysis (PDF-aligned)

このノートブックは、PDF「Momentum Strategies Across Asset Classes」の構成に沿って、
ETFユニバースに対する複数モメンタム指標の作成・比較・戦略選定を行います。

- データはCSV優先（無ければ`yfinance`）
- ロングオンリー＋キャッシュ
- レバレッジ上限: 150%
- 重み付け: リスクパリティ（Equal Risk Contribution）近似
- 選定基準: Sharpe最大

対象ETF:
`SPY, IWM, EFA, EEM, AGG, TLT, LQD, HYG, EMB, GLD, DBC, VNQ`


## 1. Setup


In [ ]:
from __future__ import annotations

import os
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize, stats

warnings.filterwarnings("ignore")
plt.style.use("ggplot")

# --- Config ---
config = {
    "tickers": ["SPY", "IWM", "EFA", "EEM", "AGG", "TLT", "LQD", "HYG", "EMB", "GLD", "DBC", "VNQ"],
    "start": "2005-01-01",
    "end": None,
    "rebalance_freq": "ME",
    "transaction_cost_bps": 10,
    "top_k": 6,
    "leverage_cap": 1.5,
    "leverage_target": 1.0,  # 필요時に変更
    "stop_loss_threshold": -0.05,
    "stop_loss_cooldown": 1,
    "seasonality_lookback_months": 120,
    "doc_vol_lookback_days": 63,
    "doc_topk": None,
    "risk_window_days": 252,
    "min_history_days": 252,
    "data_start_policy": "staggered",  # staggered|intersection|min_coverage|none
    "min_coverage_ratio": 0.8,
    "use_macro": False,
    "macro_path": "data/macro.csv",
    "macro_lookback_months": 60,
    "macro_gate_low": -0.5,
    "macro_gate_high": 0.5,
    "macro_base_strategies": ["Scorecard", "Combo_GR", "Combo_BMA", "Doc_Orig"],
    "combo_lookback_months": 60,
    "combo_min_obs": 24,
    "combo_weight_tail_months": 60,
    "gr_ridge_lambda": 1.0,
    "gr_shrink": 0.2,
    "dual_mom_signal": "ret_12_1m",
    "weights_tail_months": 24,
    "weights_top_n": 5,
}

# Output dirs (rule: figures/tables/logs under outputs/)
OUTPUT_DIR = Path("outputs")
FIG_DIR = OUTPUT_DIR / "figures"
TAB_DIR = OUTPUT_DIR / "tables"
for d in [OUTPUT_DIR, FIG_DIR, TAB_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("Config loaded:")
print(pd.Series(config))


## 2. Utility Functions


In [ ]:
# ---------- Data Loading ----------

def _read_wide_csv(path: Path) -> pd.DataFrame:
    # Detect yfinance-style multi-header CSV
    with open(path, "r") as f:
        first = f.readline().strip().split(",")
        second = f.readline().strip().split(",")
    if first and first[0] == "Price" and second and second[0] == "Ticker":
        df = pd.read_csv(path, header=[0, 1], skiprows=[2], index_col=0)
        df.index = pd.to_datetime(df.index)
        return df

    df = pd.read_csv(path)
    # try to infer date column
    date_col = None
    for c in df.columns:
        if c.lower() in ("date", "datetime", "timestamp"):
            date_col = c
            break
    if date_col is None:
        raise ValueError("Date column not found in wide CSV")
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col).sort_index()
    return df


def _read_long_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path)
    cols = {c.lower(): c for c in df.columns}
    if not {"date", "ticker", "adj_close"}.issubset(set(cols)):
        raise ValueError("Long CSV must contain date,ticker,adj_close")
    df[cols["date"]] = pd.to_datetime(df[cols["date"]])
    pivot = df.pivot(index=cols["date"], columns=cols["ticker"], values=cols["adj_close"])
    return pivot.sort_index()


def _read_per_ticker_csv(tickers: list[str], data_dir: Path) -> pd.DataFrame | None:
    frames = []
    for t in tickers:
        p = data_dir / f"{t}.csv"
        if not p.exists():
            continue
        df = pd.read_csv(p)
        # try standard column names
        date_col = None
        for c in df.columns:
            if c.lower() in ("date", "datetime", "timestamp"):
                date_col = c
                break
        if date_col is None:
            continue
        price_col = None
        for c in df.columns:
            if c.lower().replace(" ", "") in ("adjclose", "adj_close", "close"):
                price_col = c
                break
        if price_col is None:
            continue
        df[date_col] = pd.to_datetime(df[date_col])
        s = df.set_index(date_col)[price_col].rename(t)
        frames.append(s)
    if not frames:
        return None
    return pd.concat(frames, axis=1).sort_index()


def load_prices(config: dict) -> pd.DataFrame:
    data_dir = Path("data")
    wide = data_dir / "prices.csv"
    long = data_dir / "prices_long.csv"

    if wide.exists():
        print("Loading data/prices.csv")
        df = _read_wide_csv(wide)
    elif long.exists():
        print("Loading data/prices_long.csv")
        df = _read_long_csv(long)
    else:
        df = _read_per_ticker_csv(config["tickers"], data_dir)
        if df is None:
            df = None

    if df is None:
        print("CSV not found. Falling back to yfinance...")
        import yfinance as yf
        df = yf.download(
            config["tickers"],
            start=config["start"],
            end=config["end"],
            auto_adjust=False,
            progress=False,
        )
        if isinstance(df, pd.DataFrame) and "Adj Close" in df.columns:
            df = df["Adj Close"]
        elif isinstance(df, pd.Series):
            df = df.to_frame(name=config["tickers"][0])

    # Handle multi-index from yfinance-style CSV
    if isinstance(df.columns, pd.MultiIndex):
        lvl0 = df.columns.get_level_values(0)
        if "Adj Close" in lvl0:
            df = df["Adj Close"]
        elif "Close" in lvl0:
            df = df["Close"]
        df.columns.name = None

    # Standardize
    df = df.sort_index()
    df = df[config["tickers"]]
    df = df[~df.index.duplicated(keep="last")]
    df = df.astype(float)

    # Drop tickers with no data
    missing = df.columns[df.isna().all()]
    if len(missing) > 0:
        print("Warning: dropping tickers with no data:", list(missing))
        df = df.drop(columns=missing)
        config["tickers"] = [t for t in config["tickers"] if t not in missing]

    # Align start dates
    policy = config.get("data_start_policy", "intersection")
    if policy == "intersection":
        first_valid = df.apply(lambda s: s.first_valid_index())
        start = first_valid.max()
        if start is not None:
            df = df.loc[start:]
    elif policy == "staggered":
        pass
    elif policy == "min_coverage":
        ratio = config.get("min_coverage_ratio", 0.8)
        min_assets = max(1, int(np.ceil(len(df.columns) * ratio)))
        valid_count = df.notna().sum(axis=1)
        df = df.loc[valid_count >= min_assets]
    elif policy == "none":
        pass
    else:
        raise ValueError("Unknown data_start_policy")

    df = df.ffill()

    return df




def load_macro(config: dict) -> pd.DataFrame | None:
    path = Path(config.get("macro_path", "data/macro.csv"))
    if not path.exists():
        return None
    df = pd.read_csv(path)

    # infer date column
    date_col = None
    for c in df.columns:
        if c.lower() in ("date", "datetime", "timestamp"):
            date_col = c
            break
    if date_col is None:
        raise ValueError("Macro CSV must contain a date column")

    df[date_col] = pd.to_datetime(df[date_col])
    df = df.set_index(date_col).sort_index()
    df = df.apply(pd.to_numeric, errors="coerce")
    return df

# ---------- Indicators ----------

def sma(series: pd.Series, window: int) -> pd.Series:
    return series.rolling(window).mean()


def rsi(series: pd.Series, window: int = 14) -> pd.Series:
    delta = series.diff()
    up = delta.clip(lower=0)
    down = -delta.clip(upper=0)
    roll_up = up.rolling(window).mean()
    roll_down = down.rolling(window).mean()
    rs = roll_up / roll_down
    return 100 - (100 / (1 + rs))


def macd_hist(series: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9) -> pd.Series:
    ema_fast = series.ewm(span=fast, adjust=False).mean()
    ema_slow = series.ewm(span=slow, adjust=False).mean()
    macd_line = ema_fast - ema_slow
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()
    return macd_line - signal_line


def up2down(returns: pd.Series, window: int = 200) -> pd.Series:
    pos = returns.clip(lower=0).rolling(window).mean()
    neg = -returns.clip(upper=0).rolling(window).mean()
    return pos / neg


def return_to_vol(prices: pd.Series, window: int = 252) -> pd.Series:
    ret = prices.pct_change(window)
    vol = prices.pct_change().rolling(window).std() * np.sqrt(252)
    return ret / vol


# ---------- Risk Parity ----------

def _portfolio_risk_contribution(weights: np.ndarray, cov: np.ndarray) -> np.ndarray:
    port_var = weights.T @ cov @ weights
    if port_var <= 0:
        return np.zeros_like(weights)
    marginal = cov @ weights
    rc = weights * marginal / np.sqrt(port_var)
    return rc


def risk_parity_weights(cov: pd.DataFrame) -> pd.Series:
    n = cov.shape[0]
    if n == 1:
        return pd.Series([1.0], index=cov.index)

    cov_arr = cov.values

    def obj(w):
        rc = _portfolio_risk_contribution(w, cov_arr)
        target = np.mean(rc)
        return np.sum((rc - target) ** 2)

    x0 = np.ones(n) / n
    bounds = [(0.0, 1.0) for _ in range(n)]
    cons = {"type": "eq", "fun": lambda w: np.sum(w) - 1.0}

    res = optimize.minimize(obj, x0, method="SLSQP", bounds=bounds, constraints=cons, options={"maxiter": 200})
    if not res.success:
        raise RuntimeError("Risk parity optimization failed")
    return pd.Series(res.x, index=cov.index)


def inverse_vol_weights(cov: pd.DataFrame) -> pd.Series:
    vol = np.sqrt(np.diag(cov.values))
    inv = 1 / np.where(vol == 0, np.nan, vol)
    w = inv / np.nansum(inv)
    return pd.Series(w, index=cov.index)



def apply_macro_overlay(weights: pd.DataFrame, macro_gate: pd.Series | None) -> pd.DataFrame:
    if macro_gate is None:
        return weights
    gate = macro_gate.reindex(weights.index).ffill().fillna(1.0)
    return weights.mul(gate, axis=0)


def apply_stop_loss(weights: pd.DataFrame, monthly_ret: pd.DataFrame, threshold: float, cooldown: int) -> pd.DataFrame:
    # Apply stop-loss: if previous month return <= threshold, set next 'cooldown' months weight to 0.
    w = weights.copy()
    mr = monthly_ret.reindex(w.index)
    dates = list(w.index)

    blocked = pd.DataFrame(False, index=w.index, columns=w.columns)
    for i in range(1, len(dates)):
        prev = mr.iloc[i - 1]
        for col in w.columns:
            val = prev.get(col, np.nan)
            if pd.notna(val) and val <= threshold:
                end = min(len(dates), i + cooldown)
                blocked.iloc[i:end, blocked.columns.get_loc(col)] = True

    w[blocked] = 0.0
    return w

def compute_seasonality_scores(monthly_ret: pd.DataFrame, lookback_months: int) -> pd.DataFrame:
    # Compute seasonality score: average return of same calendar month over past lookback months.
    idx = monthly_ret.index
    scores = pd.DataFrame(index=idx, columns=monthly_ret.columns, dtype=float)

    for i, dt in enumerate(idx):
        start = max(0, i - lookback_months)
        hist = monthly_ret.iloc[start:i]  # exclude current month
        if hist.empty:
            continue
        same_month = hist[hist.index.month == dt.month]
        if same_month.empty:
            continue
        scores.iloc[i] = same_month.mean()

    return scores

def safe_zscore(df: pd.DataFrame) -> pd.DataFrame:
    mean = df.mean(axis=1)
    std = df.std(axis=1).replace(0.0, np.nan)
    return df.sub(mean, axis=0).div(std, axis=0)


def rank_score(df: pd.DataFrame) -> pd.DataFrame:
    return df.rank(axis=1, ascending=False, pct=True)


def doc_signals(prices: pd.DataFrame, use_log: bool = True):
    if use_log:
        mom_1m = np.log(prices / prices.shift(21))
        mom_12_1 = np.log(prices.shift(21) / prices.shift(252))
    else:
        mom_1m = prices.pct_change(21)
        mom_12_1 = prices.shift(21) / prices.shift(252) - 1.0
    ma_10_200 = prices.rolling(10).mean() / prices.rolling(200).mean() - 1.0
    return mom_1m, mom_12_1, ma_10_200


def combine_score(signals, method: str = "zscore") -> pd.DataFrame:
    mom_1m, mom_12_1, ma_10_200 = signals
    if method == "zscore":
        s1 = safe_zscore(mom_1m)
        s2 = safe_zscore(mom_12_1)
        s3 = safe_zscore(ma_10_200)
    elif method == "rank":
        s1 = rank_score(mom_1m)
        s2 = rank_score(mom_12_1)
        s3 = rank_score(ma_10_200)
    else:
        raise ValueError("Unknown method")
    return (s1 + s2 + s3) / 3.0


def rolling_vol(returns: pd.DataFrame, lookback: int = 63) -> pd.DataFrame:
    return returns.rolling(lookback).std()


def vol_adjust(score: pd.DataFrame, vol: pd.DataFrame) -> pd.DataFrame:
    return score.div(vol)


def weights_from_score(score: pd.DataFrame, mode: str, topk: int) -> pd.DataFrame:
    w = score.copy() * 0.0
    for dt in score.index:
        s = score.loc[dt].replace([np.inf, -np.inf], np.nan).fillna(0.0)
        if mode == "continuous":
            s = s.clip(lower=0.0)
            if s.sum() == 0.0:
                w.loc[dt] = 1.0 / len(s)
            else:
                w.loc[dt] = s / s.sum()
        elif mode == "topk":
            top = s.sort_values(ascending=False).head(topk)
            if top.sum() == 0.0:
                w.loc[dt] = 1.0 / len(s)
            else:
                w.loc[dt, top.index] = 1.0 / len(top)
        else:
            raise ValueError("Unknown weight mode")
    return w




def dual_mom_score(rel_score: pd.DataFrame, abs_score: pd.DataFrame) -> pd.DataFrame:
    # Relative ranking with absolute momentum gate (<=0 -> exclude)
    score = rel_score.copy()
    score = score.where(abs_score > 0)
    return score


def zscore_signal_dict(signal_dict: dict[str, pd.DataFrame], names: list[str]) -> dict[str, pd.DataFrame]:
    return {name: safe_zscore(signal_dict[name]) for name in names}


def gr_combo_score(
    signal_z: dict[str, pd.DataFrame],
    target_ret: pd.DataFrame,
    lookback_months: int,
    ridge_lambda: float,
    shrink: float,
    min_obs: int,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    # Granger-Ramanathan style regression with ridge + shrinkage
    dates = target_ret.index
    names = list(signal_z.keys())
    score = pd.DataFrame(index=dates, columns=target_ret.columns, dtype=float)
    coef_hist = pd.DataFrame(index=dates, columns=names, dtype=float)

    y_next = target_ret.shift(-1)

    for i, dt in enumerate(dates):
        if i < 2:
            continue
        start = max(0, i - lookback_months)
        idx = dates[start:i]

        X_list = []
        y_list = []
        for m in idx:
            y_m = y_next.loc[m]
            row = pd.concat({n: signal_z[n].loc[m] for n in names}, axis=1)
            df = row.copy()
            df["y"] = y_m
            df = df.dropna()
            if df.empty:
                continue
            X_list.append(df[names])
            y_list.append(df["y"])

        if not X_list:
            continue
        X = pd.concat(X_list, axis=0)
        y = pd.concat(y_list, axis=0)
        if len(y) < min_obs:
            continue

        XtX = X.T.values @ X.values
        lam = ridge_lambda * np.eye(len(names))
        try:
            beta = np.linalg.solve(XtX + lam, X.T.values @ y.values)
        except np.linalg.LinAlgError:
            beta = np.linalg.lstsq(XtX + lam, X.T.values @ y.values, rcond=None)[0]

        if shrink and shrink > 0:
            beta = (1.0 - shrink) * beta + shrink * (np.ones(len(names)) / len(names))

        coef_hist.loc[dt] = beta
        S = pd.concat({n: signal_z[n].loc[dt] for n in names}, axis=1)
        S = S[names]
        score.loc[dt] = S.dot(beta)

    return score, coef_hist


def bma_combo_score(
    signal_z: dict[str, pd.DataFrame],
    target_ret: pd.DataFrame,
    lookback_months: int,
    min_obs: int,
) -> tuple[pd.DataFrame, pd.DataFrame]:
    # Bayesian Model Averaging across single-signal models (BIC approximation)
    dates = target_ret.index
    names = list(signal_z.keys())
    score = pd.DataFrame(index=dates, columns=target_ret.columns, dtype=float)
    weight_hist = pd.DataFrame(index=dates, columns=names, dtype=float)

    y_next = target_ret.shift(-1)

    for i, dt in enumerate(dates):
        if i < 2:
            continue
        start = max(0, i - lookback_months)
        idx = dates[start:i]

        bic_vals = {}
        for name in names:
            x_list = []
            y_list = []
            for m in idx:
                x = signal_z[name].loc[m]
                y_m = y_next.loc[m]
                df = pd.concat([x, y_m], axis=1).dropna()
                if df.empty:
                    continue
                x_list.append(df.iloc[:, 0])
                y_list.append(df.iloc[:, 1])
            if not x_list:
                continue
            x_all = pd.concat(x_list)
            y_all = pd.concat(y_list)
            if len(y_all) < min_obs:
                continue

            X = np.column_stack([np.ones(len(x_all)), x_all.values])
            beta = np.linalg.lstsq(X, y_all.values, rcond=None)[0]
            resid = y_all.values - X @ beta
            rss = np.sum(resid ** 2)
            rss = max(rss, 1e-12)
            n = len(y_all)
            k = 2
            bic = n * np.log(rss / n) + k * np.log(n)
            bic_vals[name] = bic

        if not bic_vals:
            continue

        min_bic = min(bic_vals.values())
        weights = {name: np.exp(-0.5 * (bic - min_bic)) for name, bic in bic_vals.items()}
        total = sum(weights.values())
        if total == 0:
            continue
        weights = {k: v / total for k, v in weights.items()}

        weight_hist.loc[dt, list(weights.keys())] = list(weights.values())
        S = pd.concat({n: signal_z[n].loc[dt] for n in names}, axis=1)
        S = S[names]
        w = pd.Series(weights, index=names).reindex(S.columns).fillna(0.0)
        score.loc[dt] = S.dot(w)

    return score, weight_hist

# ---------- Metrics ----------

def max_drawdown(nav: pd.Series) -> tuple[float, int]:
    hwm = nav.cummax()
    dd = nav / hwm - 1.0
    max_dd = dd.min()

    # max drawdown duration
    duration = 0
    max_duration = 0
    for v in dd:
        if v < 0:
            duration += 1
        else:
            duration = 0
        max_duration = max(max_duration, duration)

    return float(max_dd), int(max_duration)


def perf_stats(returns: pd.Series, periods_per_year: int = 12) -> dict:
    r = returns.dropna()
    if r.empty:
        return {k: np.nan for k in ["AnnReturn", "CAGR", "Vol", "Sharpe", "MaxDD", "MaxDDDur", "Skew", "Kurtosis", "HitRate"]}

    ann_ret = r.mean() * periods_per_year
    cagr = (1 + r).prod() ** (periods_per_year / len(r)) - 1
    vol = r.std(ddof=1) * np.sqrt(periods_per_year)
    sharpe = ann_ret / vol if vol != 0 else np.nan

    nav = (1 + r).cumprod()
    max_dd, max_dd_dur = max_drawdown(nav)

    skew = stats.skew(r)
    kurt = stats.kurtosis(r, fisher=True)
    hit = (r >= 0).mean()

    return {
        "AnnReturn": ann_ret,
        "CAGR": cagr,
        "Vol": vol,
        "Sharpe": sharpe,
        "MaxDD": max_dd,
        "MaxDDDur": max_dd_dur,
        "Skew": skew,
        "Kurtosis": kurt,
        "HitRate": hit,
    }


## 3. Data Loading & Sanity Check


- `data_start_policy`: 価格データの開始時点を揃える方法（`staggered`/`intersection`/`min_coverage`/`none`）
- `min_coverage_ratio`: `min_coverage`時に必要な有効データ比率

- `staggered`: 早期にNaNがある資産を許容し、データが入り次第戦略に合流（行は削らない）

- 任意: `data/macro.csv` を指定するとマクロ指標を取り込み、`use_macro=True` でゲートを有効化


In [ ]:
prices = load_prices(config)
print(prices.head())
print()
print("Data coverage:")
print(prices.notna().mean().sort_values())

# Daily returns
daily_ret = prices.pct_change()

# Month-end prices & returns
month_end_prices = prices.resample(config["rebalance_freq"]).last()
monthly_ret = month_end_prices.pct_change()

print()
print("Monthly returns summary:")
print(monthly_ret.describe().T[["mean", "std", "min", "max"]])


## 3.1 Macro Data (Optional)

外生マクロ指標を読み込み、月次に整列した上でレジーム・ゲートを作成します。


In [ ]:
# Macro data (optional)
macro_df = load_macro(config)
macro_score = None
macro_gate = None

if config.get("use_macro", False) and macro_df is not None:
    macro_df = macro_df.resample(config["rebalance_freq"]).last().ffill()

    L = config.get("macro_lookback_months", 60)
    macro_mean = macro_df.rolling(L).mean()
    macro_std = macro_df.rolling(L).std()
    macro_z = (macro_df - macro_mean) / macro_std

    macro_score = macro_z.mean(axis=1)

    low = config.get("macro_gate_low", -0.5)
    high = config.get("macro_gate_high", 0.5)
    if high <= low:
        raise ValueError("macro_gate_high must be > macro_gate_low")

    macro_gate = ((macro_score - low) / (high - low)).clip(0.0, 1.0)

    # Save macro score and gate
    macro_out = pd.DataFrame({"macro_score": macro_score, "macro_gate": macro_gate})
    macro_out.to_csv(TAB_DIR / "macro_gate.csv")
    print("Saved:", TAB_DIR / "macro_gate.csv")

    fig, ax = plt.subplots(figsize=(9, 3))
    ax.plot(macro_gate.index, macro_gate.values, label="macro_gate")
    ax.set_title("Macro Gate (0-1)")
    ax.legend()
    fig.tight_layout()
    fig.savefig(FIG_DIR / "macro_gate.png", dpi=150)
    print("Saved:", FIG_DIR / "macro_gate.png")
else:
    print("Macro disabled or macro CSV not found.")


## 4. Signal Computation (PDF-aligned)


## 5. Seasonality

月別平均リターンを算出し、ヒートマップで季節性を可視化します。


In [ ]:
# Seasonality (monthly average returns)
seasonality = monthly_ret.copy()
seasonality["month"] = seasonality.index.month
seasonality_avg = seasonality.groupby("month").mean()

# Label months
month_labels = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
seasonality_avg.index = month_labels

# Save table
seasonality_avg.to_csv(TAB_DIR / "seasonality_monthly_avg.csv")
print("Saved:", TAB_DIR / "seasonality_monthly_avg.csv")

# Plot heatmap
fig, ax = plt.subplots(figsize=(9, 4))
im = ax.imshow(seasonality_avg.values, aspect="auto", cmap="coolwarm")
ax.set_xticks(range(len(seasonality_avg.columns)))
ax.set_xticklabels(seasonality_avg.columns, rotation=45, ha="right")
ax.set_yticks(range(len(seasonality_avg.index)))
ax.set_yticklabels(seasonality_avg.index)
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
ax.set_title("Seasonality: Monthly Average Returns")
fig.tight_layout()
fig_path = FIG_DIR / "seasonality_heatmap.png"
fig.savefig(fig_path, dpi=150)
print("Saved:", fig_path)

# Seasonality score for strategy (rolling same-month average)
seasonality_score = compute_seasonality_scores(monthly_ret, config["seasonality_lookback_months"])
print("Seasonality score ready:", seasonality_score.dropna(how="all").shape)


In [ ]:
# Daily-based signals, then sampled at month-end
freq = config["rebalance_freq"]
me = month_end_prices.index

# returns over trading-day windows
ret_1m = prices.pct_change(21).resample(freq).last()
ret_3m = prices.pct_change(63).resample(freq).last()
ret_6m = prices.pct_change(126).resample(freq).last()
ret_12m = prices.pct_change(252).resample(freq).last()
ret_12_1m = (prices.shift(21) / prices.shift(252) - 1).resample(freq).last()

sma200 = (prices / sma(prices, 200) - 1).resample(freq).last()
ma_5_100 = (sma(prices, 5) / sma(prices, 100) - 1).resample(freq).last()
ma_10_200 = (sma(prices, 10) / sma(prices, 200) - 1).resample(freq).last()
ma_50_200 = (sma(prices, 50) / sma(prices, 200) - 1).resample(freq).last()

rsi_14 = prices.apply(rsi).resample(freq).last()
macd_hist_m = prices.apply(macd_hist).resample(freq).last()
up2down_200 = daily_ret.apply(lambda s: up2down(s, 200)).resample(freq).last()
ret_to_vol_3m = prices.apply(lambda s: return_to_vol(s, 63)).resample(freq).last()
ret_to_vol_12m = prices.apply(lambda s: return_to_vol(s, 252)).resample(freq).last()

ret_6m_minus_2w = (prices.pct_change(126) - prices.pct_change(10)).resample(freq).last()

signals = {
    "ret_1m": ret_1m,
    "ret_3m": ret_3m,
    "ret_6m": ret_6m,
    "ret_12m": ret_12m,
    "ret_12_1m": ret_12_1m,
    "sma200": sma200,
    "ma_5_100": ma_5_100,
    "ma_10_200": ma_10_200,
    "ma_50_200": ma_50_200,
    "rsi_14": rsi_14,
    "macd_hist": macd_hist_m,
    "up2down_200": up2down_200,
    "ret_to_vol_3m": ret_to_vol_3m,
    "ret_to_vol_12m": ret_to_vol_12m,
    "ret_6m_minus_2w": ret_6m_minus_2w,
}

print("Signals computed:", list(signals.keys()))

# Doc Benchmark variants (signals -> score)
# Use month-end sampling of daily signals
sig_log = doc_signals(prices, use_log=True)
sig_pct = doc_signals(prices, use_log=False)

score_log_z = combine_score(sig_log, method="zscore").resample(config["rebalance_freq"]).last()
score_pct_z = combine_score(sig_pct, method="zscore").resample(config["rebalance_freq"]).last()
score_log_rank = combine_score(sig_log, method="rank").resample(config["rebalance_freq"]).last()

vol_63d = rolling_vol(prices.pct_change(), lookback=config["doc_vol_lookback_days"]).resample(config["rebalance_freq"]).last()

score_log_z_vol = vol_adjust(score_log_z, vol_63d)
score_pct_z_vol = vol_adjust(score_pct_z, vol_63d)
score_log_rank_vol = vol_adjust(score_log_rank, vol_63d)

# For no-vol-adjust variant
score_log_z_novol = score_log_z


In [ ]:
# Advanced signal combinations (Dual Momentum / GR / BMA)
combo_signals = [
    "ret_6m",
    "ret_12m",
    "ret_to_vol_3m",
    "ret_to_vol_12m",
    "sma200",
    "ma_5_100",
    "ma_10_200",
    "ret_6m_minus_2w",
]

combo_z = zscore_signal_dict(signals, combo_signals)

# Dual Momentum: relative ranking with absolute momentum gate
_dm_base = signals[config["dual_mom_signal"]]
dual_mom_score_df = dual_mom_score(_dm_base, _dm_base)

# Granger-Ramanathan (ridge + shrinkage) combo
combo_gr_score, combo_gr_weights = gr_combo_score(
    combo_z,
    monthly_ret,
    config["combo_lookback_months"],
    config["gr_ridge_lambda"],
    config["gr_shrink"],
    config["combo_min_obs"],
)

# Bayesian Model Averaging (BIC-weighted single-signal models)
combo_bma_score, combo_bma_weights = bma_combo_score(
    combo_z,
    monthly_ret,
    config["combo_lookback_months"],
    config["combo_min_obs"],
)

print("Advanced combo signals ready:", ["DualMom", "Combo_GR", "Combo_BMA"])


## 6. Strategy Construction (Long-only + Cash)


### 各手法の詳細説明（論文記載レベル・拡張）

以下では、資産 $i$ の価格を $P_{t,i}$、月次リターンを $r_{t,i}$、日次リターンを $r^{(d)}_{t,i}$ とします。月末リバランスで、当月のシグナルから翌月の保有を決定します（実装上は $w_{t-1}$ を $t$ のリターンに適用）。

**共通の前処理と記法**  
- 単純リターン: $r_{t,i} = \frac{P_{t,i}}{P_{t-1,i}} - 1$  
- $k$ ヶ月モメンタム: $m^{(k)}_{t,i} = \frac{P_{t,i}}{P_{t-k,i}} - 1$  
- 12-1モメンタム: $m^{(12-1)}_{t,i} = \frac{P_{t-1,i}}{P_{t-12,i}} - 1$  
- 日次ボラティリティ: $\sigma_{t,i} = \text{std}(r^{(d)}_{t-L+1:t,i}) \cdot \sqrt{252}$  
- クロスセクションZスコア: $z_{t,i} = \frac{x_{t,i} - \bar{x}_t}{s_t}$  
- ランクスコア: $\text{rank}_{t,i} = \frac{\text{rank}(x_{t,i})}{N}$（大きいほど良い）

**ポートフォリオ・リターンとコスト**  
- 月次総リターン: $R_t = \sum_i w_{t-1,i}\, r_{t,i}$  
- ターンオーバー: $\tau_t = \sum_i |w_{t,i}-w_{t-1,i}|$  
- 取引コスト: $c_t = \tau_{t-1}\cdot \text{bps}/10000$  
- 純リターン: $R_t^{net} = R_t - c_t$  

---

**TSM（Absolute Momentum）**  
定義: 単一資産のトレンドに基づきロング/キャッシュを切替。  
スコア: $s_{t,i} = m^{(k)}_{t,i}$（$k \in \{1,3,6,12\}$）  
選択: $s_{t,i} > 0$ のみ投資、その他はキャッシュ。  
重み: 選択集合 $S_t$ に対してリスクパリティで配分（$\sum_i w_{t,i}=1$）。  
特徴: トレンド転換でドローダウンが出やすく、短期ほど回転率が高い。

**CSM（Relative Momentum）**  
定義: 同一時点の相対順位でTop-Kを選択。  
スコア: $s_{t,i} = m^{(k)}_{t,i}$  
選択: 上位 $K$ 資産のみロング。  
重み: 選択集合 $S_t$ に対してリスクパリティで配分。  
特徴: 全体下落局面でもTop-Kが負となる可能性がある。

**Risk-Adjusted CSM**  
定義: リターンをボラで割って相対的な強さを評価。  
スコア: $s_{t,i} = \frac{m^{(12)}_{t,i}}{\sigma_{t,i}}$  
選択: 上位 $K$ をロング。  
特徴: ボラ推定窓に依存し、短期ノイズで順位が変動しやすい。

**Scorecard（複合モメンタム）**  
定義: 複数シグナルのランク平均で合成スコアを作る。  
指標: $\{m^{(6)}, m^{(12)}, m^{(3)}/\sigma, m^{(12)}/\sigma, \text{SMA200}, \text{MA}_{5/100}, \text{MA}_{10/200}, m^{(6)}-m^{(2w)}\}$  
合成: $s_{t,i} = \frac{1}{J}\sum_{j=1}^J \text{rank}_{t,i}^{(j)}$  
選択: 上位 $K$ をロング（Top-K）。  
特徴: 単一指標依存を避け、信号の頑健性を高める。

**DTF_simple（信号分散の簡易版）**  
定義: 短期・中期・トレンドの3信号を平均。  
合成: $s_{t,i} = \frac{1}{3}(\text{rank}(m^{(1)}_{t,i}) + \text{rank}(m^{(12)}_{t,i}) + \text{rank}(\text{MA}_{10/200,t,i}))$  
選択: 上位 $K$ をロング。  
特徴: 時間軸分散でトレンド検出の安定性を高める。

**Seasonality戦略**  
定義: 同じ暦月の過去平均リターンで順位付け。  
スコア: $s_{t,i} = \frac{1}{L}\sum_{\ell=1}^L r_{t-12\ell,i}$  
選択: 上位 $K$ をロング。  
注意点: 構造変化に弱く、継続性は保証されない。

**Stop-loss（-5%）**  
定義: 前月リターンが閾値以下なら翌月はキャッシュ。  
条件: $r_{t-1,i} \le -5\%$ の場合、$w_{t,i}=0$（クールダウン1ヶ月）。  
注意点: 反転局面で上昇を取り逃がすリスクがある。

**EQWベンチマーク**  
定義: 毎月末に均等ウェイトで再配分。  
重み: $w_{t,i}=1/N$。

**リスクパリティ（Equal Risk Contribution）**  
目的: 各資産のリスク寄与を等しくする。  
条件: $RC_i = \frac{w_i (\Sigma w)_i}{\sqrt{w^T\Sigma w}}$ が全 $i$ で等しい。  
実装: 過去 `risk_window_days` の日次共分散 $\Sigma$ を用いて最適化。

**Dual Momentum（相対×絶対ゲーティング）**  
定義: 相対順位でTop-Kを選びつつ、絶対モメンタムが正の資産のみ採用。  
ゲート: $m^{(12-1)}_{t,i} > 0$ を満たす資産のみランク計算。  
選択: 上位 $K$ をロング。  
特徴: 市場全体が弱い局面では保有が減少し、キャッシュ比率が高まりやすい。

**GR合成（回帰＋縮小）**  
定義: 過去の予測力に基づき複数シグナルを線形結合。  
モデル: $r_{t+1,i} = \alpha + \sum_{j=1}^J \beta_j z^{(j)}_{t,i} + \varepsilon_{t,i}$  
推定: リッジ回帰 $\min_\beta \|y-X\beta\|^2 + \lambda \|\beta\|^2$ を解き、等重みへ縮小。  
合成: $s_{t,i} = \sum_j \beta_j z^{(j)}_{t,i}$。

**BMA合成（単一モデルのベイズ平均）**  
定義: 各シグナル単独モデルの情報量（BIC）から重みを算定し合成。  
重み: $w_j \propto \exp\{-0.5(\text{BIC}_j-\min \text{BIC})\}$  
合成: $s_{t,i} = \sum_j w_j z^{(j)}_{t,i}$。

**Doc Benchmark（原典ベースのマルチシグナル）**  
定義: 1M, 12-1M, MA10/200 の3シグナルをZスコア合成し、ボラ調整で強度を補正。  
シグナル: $z_{t,i}^{(j)}$ を平均して $s_{t,i}$ を作り、$s'_{t,i}=s_{t,i}/\sigma_{t,i}$。  
重み: $w_{t,i} \propto \max(s'_{t,i},0)$ の連続ウェイト。  
差分比較: log↔pct、Z↔rank、ボラ調整有無、連続↔Top-Kをアブレーションで検証。


**Macro Overlay（外生指標によるゲート）**  
定義: マクロ指標を標準化し、リスク・オン/オフのゲート $g_t\in[0,1]$ を作成して戦略ウェイトをスケールする。  
スコア: $g_t = \text{clip}\left(\frac{\bar{z}_t-\ell}{h-\ell},0,1\right)$、$\bar{z}_t=\frac{1}{M}\sum_{m=1}^M z_{t,m}$  
適用: $w^{macro}_{t,i}=g_t\, w_{t,i}$（$g_t=0$ でキャッシュ）。  
目的: マクロ環境に応じてエクスポージャーを調整し、下方リスクを抑制。


### Doc系戦略のパフォーマンスが良好になりやすい理由（PDFの記述に基づく補足）

- PDFでは、複数のトレンド・シグナルを組み合わせた分散（Diversified Trend-Following）が単一シグナルよりも安定しやすいと示唆されている。Doc系は1M・12-1M・MA10/200の3本を合成してこの分散効果を取り込む。  
- リスク調整（ボラティリティ・スケーリング）は、同じモメンタムでもリターン増加とリスク低下の両面で改善し得ると報告されている。Doc系の`score_adj = score / vol_63d`はこの考え方に沿う。  
- Long-onlyモメンタムはショート脚のドラッグを避けられるため、ロング/ショートよりSharpeが高いケースが多いと述べられている。Doc系は連続ウェイトでロングのみを構成する。  
- 6〜12か月のトレンド指標は短期よりもリスク調整後の成績が良い傾向があり、回転率低下によるコスト抑制も要因として挙げられている。Doc系の12-1Mはこの性質を取り込み、1Mは短期反応を補完する。  
- 以上を総合すると、Doc系は「複数シグナル分散 + リスク調整 + ロングのみ + 強度反映（連続ウェイト）」というPDFの推奨要素を同時に満たすため、結果としてリスク調整後パフォーマンスが良好になりやすい。  


In [ ]:
# Helper to build weights

def build_weights(score_df: pd.DataFrame, mode: str, name: str) -> pd.DataFrame:
    tickers = config["tickers"]
    w = pd.DataFrame(0.0, index=score_df.index, columns=tickers)

    for dt in score_df.index:
        score = score_df.loc[dt].dropna()
        if score.empty:
            continue

        if mode == "absolute":
            eligible = score[score > 0].index
        elif mode == "relative":
            eligible = score.sort_values(ascending=False).head(config["top_k"]).index
        else:
            raise ValueError("Unknown mode")

        if len(eligible) == 0:
            continue

        # Covariance window
        hist = daily_ret.loc[:dt].tail(config["risk_window_days"])
        hist = hist[eligible].dropna(how="all")
        cov = hist.cov()
        cov = cov.loc[eligible, eligible]
        cov = cov.dropna(axis=0, how="all").dropna(axis=1, how="all")
        if cov.shape[0] == 0:
            continue
        cov = cov.fillna(0.0)
        cov = cov + np.eye(cov.shape[0]) * 1e-6

        try:
            weights = risk_parity_weights(cov)
        except Exception:
            try:
                weights = inverse_vol_weights(cov)
            except Exception:
                weights = pd.Series(1.0 / len(eligible), index=eligible)

        # scale to leverage target (cap)
        leverage_target = min(config["leverage_target"], config["leverage_cap"])
        weights = weights / weights.sum() * leverage_target

        w.loc[dt, weights.index] = weights.values

    return w


# --- Strategy definitions ---
strategies = {}

# Absolute Momentum (TSM)
strategies["TSM_12M"] = {"scores": signals["ret_12m"], "mode": "absolute"}
strategies["TSM_6M"] = {"scores": signals["ret_6m"], "mode": "absolute"}
strategies["TSM_3M"] = {"scores": signals["ret_3m"], "mode": "absolute"}
strategies["TSM_1M"] = {"scores": signals["ret_1m"], "mode": "absolute"}
strategies["TSM_12_1M"] = {"scores": signals["ret_12_1m"], "mode": "absolute"}

# Relative Momentum (CSM)
strategies["CSM_12M"] = {"scores": signals["ret_12m"], "mode": "relative"}
strategies["CSM_RiskAdj_12M"] = {"scores": signals["ret_to_vol_12m"], "mode": "relative"}

# Scorecard (PDF-aligned 8 signals)
score_signals = combo_signals

score_df = None
for s in score_signals:
    df = signals[s]
    rank = df.rank(axis=1, ascending=False)
    score_df = rank if score_df is None else score_df.add(rank, fill_value=0)
score_df = score_df / len(score_signals)

strategies["Scorecard"] = {"scores": score_df, "mode": "relative"}

# DTF simple (1M, 12M, MA cross)
_dt = signals["ret_1m"].rank(axis=1, ascending=False)
_dt = _dt.add(signals["ret_12m"].rank(axis=1, ascending=False), fill_value=0)
_dt = _dt.add(signals["ma_10_200"].rank(axis=1, ascending=False), fill_value=0)
_dt = _dt / 3
strategies["DTF_simple"] = {"scores": _dt, "mode": "relative"}

# Seasonality strategy (TopK by same-month avg return)
strategies["Seasonality"] = {"scores": seasonality_score, "mode": "relative"}

# Advanced signal combinations
strategies["DualMom"] = {"scores": dual_mom_score_df, "mode": "relative"}
strategies["Combo_GR"] = {"scores": combo_gr_score, "mode": "relative"}
strategies["Combo_BMA"] = {"scores": combo_bma_score, "mode": "relative"}

# Doc Benchmark variants



# Build weights

weights = {}
for name, spec in strategies.items():
    w = build_weights(spec["scores"], spec["mode"], name)
    weights[name] = w

print("Strategies built:", list(weights.keys()))

# Macro overlay variants (optional)
macro_weights = {}
if config.get("use_macro", False) and macro_gate is not None:
    base_list = config.get("macro_base_strategies", [])
    for base in base_list:
        if base in weights:
            macro_weights[f"{base}_Macro"] = apply_macro_overlay(weights[base], macro_gate)
    if macro_weights:
        weights.update(macro_weights)
        print("Macro overlay strategies added:", list(macro_weights.keys()))
else:
    print("Macro overlay disabled or macro gate not available.")



## 7. Stop-loss (Monthly -5% block)

前月リターンが-5%以下の銘柄は翌月キャッシュ化します。


In [ ]:
# Doc Benchmark variants (weights)
_doc_topk = config["doc_topk"] or config["top_k"]

doc_weights_orig = weights_from_score(score_log_z_vol, mode="continuous", topk=_doc_topk)
doc_weights_pct = weights_from_score(score_pct_z_vol, mode="continuous", topk=_doc_topk)
doc_weights_rank = weights_from_score(score_log_rank_vol, mode="continuous", topk=_doc_topk)
doc_weights_novol = weights_from_score(score_log_z_novol, mode="continuous", topk=_doc_topk)
doc_weights_topk = weights_from_score(score_log_z_vol, mode="topk", topk=_doc_topk)

weights["Doc_Orig"] = doc_weights_orig
weights["Doc_Pct"] = doc_weights_pct
weights["Doc_Rank"] = doc_weights_rank
weights["Doc_NoVolAdj"] = doc_weights_novol
weights["Doc_TopK"] = doc_weights_topk


In [ ]:
# Apply stop-loss to existing strategies
stop_threshold = config["stop_loss_threshold"]
stop_cooldown = config["stop_loss_cooldown"]

weights_sl = {}
for name, w in weights.items():
    weights_sl[f"{name}_SL"] = apply_stop_loss(w, monthly_ret, stop_threshold, stop_cooldown)

weights.update(weights_sl)
print("Stop-loss strategies added:", list(weights_sl.keys()))


## 8. Backtest


In [ ]:
# Backtest returns
results = {}
turnovers = {}

# EQW benchmark (monthly equal weight rebalance)
N = len(config["tickers"])
w_eqw = pd.DataFrame(1.0 / N, index=monthly_ret.index, columns=config["tickers"])

all_weights = dict(weights)
all_weights["EQW"] = w_eqw

for name, w in all_weights.items():
    w = w.reindex(monthly_ret.index).fillna(0.0)
    gross = (w.shift(1) * monthly_ret).sum(axis=1)
    turnover = w.diff().abs().sum(axis=1)
    cost = turnover.shift(1) * (config["transaction_cost_bps"] / 10000.0)
    net = gross - cost

    results[name] = net
    turnovers[name] = turnover

returns_df = pd.DataFrame(results)

# Performance table
perf = pd.DataFrame({name: perf_stats(returns_df[name]) for name in returns_df.columns}).T
perf = perf.sort_values("Sharpe", ascending=False)

perf.to_csv(TAB_DIR / "performance_summary.csv")
print("Saved:", TAB_DIR / "performance_summary.csv")
perf


### 各手法のウェイト推移の出力

Sharpe上位の戦略（上位N）+ EQW の月次ウェイトをCSVと図で保存します。


In [ ]:
# Export weight histories for top strategies
import re

weight_paths = []

top_n = config.get("weights_top_n", 5)
selected = list(perf.head(top_n).index)
if "EQW" not in selected:
    selected.append("EQW")

for name in selected:
    w = all_weights.get(name)
    if w is None:
        continue
    w = w.reindex(monthly_ret.index).fillna(0.0)

    safe = re.sub(r"[^A-Za-z0-9_-]+", "_", name)
    csv_path = TAB_DIR / f"weights_{safe}.csv"
    w.to_csv(csv_path)

    tail = config.get("weights_tail_months", 24)
    w_plot = w.tail(tail) if tail else w
    w_plot = w_plot.fillna(0.0)

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.stackplot(w_plot.index, w_plot.T, labels=w_plot.columns)
    ax.set_title(f"Weights: {name} (last {len(w_plot)} months)")
    ax.legend(ncol=2, fontsize=7, loc="upper left")
    fig.tight_layout()
    fig_path = FIG_DIR / f"weights_{safe}.png"
    fig.savefig(fig_path, dpi=150)
    plt.close(fig)

    weight_paths.append((csv_path, fig_path))

print(f"Saved weights for {len(weight_paths)} strategies")


## 9. Visualization & Output


In [ ]:
# Cumulative returns plot (Top 5 Sharpe + EQW)
perf_sorted = perf.copy()
cols = list(perf_sorted.head(5).index)
if "EQW" not in cols:
    cols.append("EQW")

cum = (1 + returns_df[cols]).cumprod()
fig, ax = plt.subplots(figsize=(10, 5))
for col in cum.columns:
    ax.plot(cum.index, cum[col], label=col)
ax.set_title("Cumulative Returns (Top 5 Sharpe + EQW)")
ax.legend(ncol=2, fontsize=8)
fig.tight_layout()
fig_path = FIG_DIR / "cumulative_returns.png"
fig.savefig(fig_path, dpi=150)
print("Saved:", fig_path)

# Correlation heatmap (all strategies)
corr = returns_df.corr()
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(corr.values, cmap="coolwarm", vmin=-1, vmax=1)
ax.set_xticks(range(len(corr.columns)))
ax.set_yticks(range(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=45, ha="right")
ax.set_yticklabels(corr.columns)
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
ax.set_title("Strategy Correlation")
fig.tight_layout()
fig_path = FIG_DIR / "strategy_correlation.png"
fig.savefig(fig_path, dpi=150)
print("Saved:", fig_path)

# Save correlation table
corr.to_csv(TAB_DIR / "strategy_correlation.csv")


In [ ]:
# Cumulative returns vs EQW for each strategy
import re

eqw_col = "EQW"
if eqw_col not in returns_df.columns:
    print("EQW not found; skipping EQW comparison plots.")
else:
    out_paths = []
    for strat in returns_df.columns:
        if strat == eqw_col:
            continue
        cols = [strat, eqw_col]
        cum = (1 + returns_df[cols]).cumprod()
        fig, ax = plt.subplots(figsize=(8, 4))
        for col in cum.columns:
            ax.plot(cum.index, cum[col], label=col)
        ax.set_title(f"Cumulative Returns: {strat} vs EQW")
        ax.legend(fontsize=8)
        fig.tight_layout()
        safe = re.sub(r"[^A-Za-z0-9_-]+", "_", strat)
        fig_path = FIG_DIR / f"cumret_vs_eqw_{safe}.png"
        fig.savefig(fig_path, dpi=150)
        plt.close(fig)
        out_paths.append(fig_path)
    print(f"Saved {len(out_paths)} EQW comparison plots to {FIG_DIR}")


### 合成手法の感度分析と重み推移

GR合成の学習窓・縮小率の感度を確認し、GR/BMAのシグナル重み推移を可視化します。


In [ ]:
# Sensitivity analysis for GR combo (lookback x shrink)
lookbacks = [36, 60, 84]
shrinks = [0.0, 0.2, 0.5]

records = []

def backtest_from_weights(w: pd.DataFrame, monthly_ret: pd.DataFrame, bps: float) -> pd.Series:
    w = w.reindex(monthly_ret.index).fillna(0.0)
    gross = (w.shift(1) * monthly_ret).sum(axis=1)
    turnover = w.diff().abs().sum(axis=1)
    cost = turnover.shift(1) * (bps / 10000.0)
    return gross - cost

for lb in lookbacks:
    for sh in shrinks:
        score, _ = gr_combo_score(
            combo_z,
            monthly_ret,
            lb,
            config["gr_ridge_lambda"],
            sh,
            config["combo_min_obs"],
        )
        w = build_weights(score, mode="relative", name=f"Combo_GR_L{lb}_S{sh}")
        ret = backtest_from_weights(w, monthly_ret, config["transaction_cost_bps"])
        stats_row = perf_stats(ret)
        stats_row.update({"lookback": lb, "shrink": sh})
        records.append(stats_row)

sens_df = pd.DataFrame(records)
sens_df = sens_df[["lookback", "shrink", "Sharpe", "AnnReturn", "Vol", "MaxDD"]]

sens_path = TAB_DIR / "combo_gr_sensitivity.csv"
sens_df.to_csv(sens_path, index=False)
print("Saved:", sens_path)

# Heatmap (Sharpe)
heat = sens_df.pivot(index="lookback", columns="shrink", values="Sharpe")
fig, ax = plt.subplots(figsize=(6, 4))
im = ax.imshow(heat.values, cmap="viridis", aspect="auto")
ax.set_xticks(range(len(heat.columns)))
ax.set_yticks(range(len(heat.index)))
ax.set_xticklabels(heat.columns)
ax.set_yticklabels(heat.index)
ax.set_xlabel("shrink")
ax.set_ylabel("lookback months")
ax.set_title("GR Combo Sensitivity (Sharpe)")
fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
fig.tight_layout()
fig_path = FIG_DIR / "combo_gr_sensitivity_sharpe.png"
fig.savefig(fig_path, dpi=150)
print("Saved:", fig_path)

# Plot GR/BMA signal weights (tail)
def plot_signal_weights(weight_df: pd.DataFrame, name: str) -> None:
    w = weight_df.copy().dropna(how="all")
    if w.empty:
        print(f"No weights for {name}")
        return
    tail = config.get("combo_weight_tail_months", 60)
    if tail:
        w = w.tail(tail)
    w = w.fillna(0.0)

    csv_path = TAB_DIR / f"{name}_weights.csv"
    w.to_csv(csv_path)
    print("Saved:", csv_path)

    fig, ax = plt.subplots(figsize=(9, 4))
    ax.stackplot(w.index, w.T, labels=w.columns)
    ax.set_title(f"{name} signal weights (last {len(w)} months)")
    ax.legend(ncol=2, fontsize=7, loc="upper left")
    fig.tight_layout()
    fig_path = FIG_DIR / f"{name}_weights.png"
    fig.savefig(fig_path, dpi=150)
    plt.close(fig)
    print("Saved:", fig_path)

plot_signal_weights(combo_gr_weights, "combo_gr")
plot_signal_weights(combo_bma_weights, "combo_bma")


In [ ]:
# Doc Benchmark variants comparison outputs
variants = ["Doc_Orig", "Doc_Pct", "Doc_Rank", "Doc_NoVolAdj", "Doc_TopK"]

variants_ret = returns_df[variants]

# Summary table
variants_perf = perf.loc[variants]
variants_perf.to_csv(TAB_DIR / "doc_benchmark_variants_summary.csv")
print("Saved:", TAB_DIR / "doc_benchmark_variants_summary.csv")

# Cumulative returns
cum = (1 + variants_ret).cumprod()
fig, ax = plt.subplots(figsize=(9, 4))
for col in cum.columns:
    ax.plot(cum.index, cum[col], label=col)
ax.set_title("Doc Benchmark Variants: Cumulative Returns")
ax.legend(ncol=2, fontsize=8)
fig.tight_layout()
fig.savefig(FIG_DIR / "doc_benchmark_variants_cumret.png", dpi=150)
print("Saved:", FIG_DIR / "doc_benchmark_variants_cumret.png")

# Drawdown
peak = cum.cummax()
dd = cum / peak - 1
fig, ax = plt.subplots(figsize=(9, 3))
for col in dd.columns:
    ax.plot(dd.index, dd[col], label=col)
ax.set_title("Doc Benchmark Variants: Drawdown")
ax.legend(ncol=2, fontsize=8)
fig.tight_layout()
fig.savefig(FIG_DIR / "doc_benchmark_variants_drawdown.png", dpi=150)
print("Saved:", FIG_DIR / "doc_benchmark_variants_drawdown.png")

# Annual summary
out = []
for col in variants_ret.columns:
    r = variants_ret[col]
    for year, yr in r.groupby(r.index.year):
        if len(yr) == 0:
            continue
        ann_return = (1 + yr).prod() - 1
        ann_vol = (12.0 ** 0.5) * yr.std()
        sharpe = (12.0 ** 0.5) * yr.mean() / yr.std() if yr.std() != 0 else 0.0
        cumy = (1 + yr).cumprod()
        dd_y = cumy / cumy.cummax() - 1
        out.append([col, year, ann_return, ann_vol, sharpe, float(dd_y.min())])

annual = pd.DataFrame(out, columns=["strategy", "year", "annual_return", "annual_vol", "sharpe", "max_drawdown"]).set_index(["strategy", "year"])
annual.to_csv(TAB_DIR / "doc_benchmark_variants_annual_summary.csv")
print("Saved:", TAB_DIR / "doc_benchmark_variants_annual_summary.csv")

# Doc Benchmark variants comparison outputs (Stop-loss variants)
variants_sl = [f"{v}_SL" for v in variants if f"{v}_SL" in returns_df.columns]

if variants_sl:
    variants_sl_ret = returns_df[variants_sl]
    variants_sl_perf = perf.loc[variants_sl]

    variants_sl_perf.to_csv(TAB_DIR / "doc_benchmark_variants_sl_summary.csv")
    print("Saved:", TAB_DIR / "doc_benchmark_variants_sl_summary.csv")

    cum_sl = (1 + variants_sl_ret).cumprod()
    fig, ax = plt.subplots(figsize=(9, 4))
    for col in cum_sl.columns:
        ax.plot(cum_sl.index, cum_sl[col], label=col)
    ax.set_title("Doc Benchmark Variants (SL): Cumulative Returns")
    ax.legend(ncol=2, fontsize=8)
    fig.tight_layout()
    fig.savefig(FIG_DIR / "doc_benchmark_variants_sl_cumret.png", dpi=150)
    print("Saved:", FIG_DIR / "doc_benchmark_variants_sl_cumret.png")

    peak_sl = cum_sl.cummax()
    dd_sl = cum_sl / peak_sl - 1
    fig, ax = plt.subplots(figsize=(9, 3))
    for col in dd_sl.columns:
        ax.plot(dd_sl.index, dd_sl[col], label=col)
    ax.set_title("Doc Benchmark Variants (SL): Drawdown")
    ax.legend(ncol=2, fontsize=8)
    fig.tight_layout()
    fig.savefig(FIG_DIR / "doc_benchmark_variants_sl_drawdown.png", dpi=150)
    print("Saved:", FIG_DIR / "doc_benchmark_variants_sl_drawdown.png")

    out_sl = []
    for col in variants_sl_ret.columns:
        r = variants_sl_ret[col]
        for year, yr in r.groupby(r.index.year):
            if len(yr) == 0:
                continue
            ann_return = (1 + yr).prod() - 1
            ann_vol = (12.0 ** 0.5) * yr.std()
            sharpe = (12.0 ** 0.5) * yr.mean() / yr.std() if yr.std() != 0 else 0.0
            cumy = (1 + yr).cumprod()
            dd_y = cumy / cumy.cummax() - 1
            out_sl.append([col, year, ann_return, ann_vol, sharpe, float(dd_y.min())])

    annual_sl = pd.DataFrame(out_sl, columns=["strategy", "year", "annual_return", "annual_vol", "sharpe", "max_drawdown"]).set_index(["strategy", "year"])
    annual_sl.to_csv(TAB_DIR / "doc_benchmark_variants_sl_annual_summary.csv")
    print("Saved:", TAB_DIR / "doc_benchmark_variants_sl_annual_summary.csv")


In [ ]:
# Doc Benchmark difference analysis (factor contributions)
base = variants_perf.loc["Doc_Orig"]

compare_keys = ["Doc_Pct", "Doc_Rank", "Doc_NoVolAdj", "Doc_TopK"]
doc_diff = variants_perf.loc[compare_keys][["AnnReturn", "Vol", "Sharpe", "MaxDD"]].sub(base[["AnnReturn", "Vol", "Sharpe", "MaxDD"]])

doc_diff.index.name = "variant"

# Identify largest absolute Sharpe impact
largest_sharpe_impact = doc_diff["Sharpe"].abs().sort_values(ascending=False)
if not largest_sharpe_impact.empty:
    doc_largest_key = largest_sharpe_impact.index[0]
    doc_largest_val = doc_diff.loc[doc_largest_key, "Sharpe"]
else:
    doc_largest_key, doc_largest_val = None, None

# Save
DOC_DIFF_PATH = TAB_DIR / "doc_benchmark_factor_contrib.csv"
doc_diff.to_csv(DOC_DIFF_PATH)
print("Saved:", DOC_DIFF_PATH)

# Stop-loss variants factor contribution (vs Doc_Orig_SL)
variants_sl = [f"{v}_SL" for v in ["Doc_Orig", "Doc_Pct", "Doc_Rank", "Doc_NoVolAdj", "Doc_TopK"] if f"{v}_SL" in perf.index]
if variants_sl:
    base_sl = perf.loc["Doc_Orig_SL"]
    compare_keys_sl = [v for v in variants_sl if v != "Doc_Orig_SL"]
    doc_diff_sl = perf.loc[compare_keys_sl][["AnnReturn", "Vol", "Sharpe", "MaxDD"]].sub(base_sl[["AnnReturn", "Vol", "Sharpe", "MaxDD"]])
    doc_diff_sl.index.name = "variant"
    DOC_DIFF_SL_PATH = TAB_DIR / "doc_benchmark_factor_contrib_sl.csv"
    doc_diff_sl.to_csv(DOC_DIFF_SL_PATH)
    print("Saved:", DOC_DIFF_SL_PATH)


In [ ]:
# Doc variants + Scorecard comparison (same plot)
compare_cols = ["Scorecard", "Doc_Orig", "Doc_Pct", "Doc_Rank", "Doc_NoVolAdj", "Doc_TopK"]
compare_cols = [c for c in compare_cols if c in returns_df.columns]

cum = (1 + returns_df[compare_cols]).cumprod()
fig, ax = plt.subplots(figsize=(10, 5))
for col in cum.columns:
    ax.plot(cum.index, cum[col], label=col)
ax.set_title("Scorecard vs Doc Variants: Cumulative Returns")
ax.legend(ncol=2, fontsize=8)
fig.tight_layout()
fig_path = FIG_DIR / "doc_vs_scorecard_cumret.png"
fig.savefig(fig_path, dpi=150)
print("Saved:", fig_path)

peak = cum.cummax()
dd = cum / peak - 1
fig, ax = plt.subplots(figsize=(10, 4))
for col in dd.columns:
    ax.plot(dd.index, dd[col], label=col)
ax.set_title("Scorecard vs Doc Variants: Drawdown")
ax.legend(ncol=2, fontsize=8)
fig.tight_layout()
fig_path = FIG_DIR / "doc_vs_scorecard_drawdown.png"
fig.savefig(fig_path, dpi=150)
print("Saved:", fig_path)


## 10. Strategy Selection (Sharpe Max)


In [ ]:
best = perf.head(3)
print("Top 3 strategies by Sharpe:")
print(best[["Sharpe", "AnnReturn", "MaxDD", "Vol"]])

best.to_csv(TAB_DIR / "top3_strategies.csv")
print("Saved:", TAB_DIR / "top3_strategies.csv")


## 11. レポート章立て（PDF構成準拠）

以下はPDFの章構成に沿ったレポート本文（Notebook内表示用）です。実データ結果に基づく要約は次セルで自動生成します。


In [ ]:
from IPython.display import Markdown, display

# Dynamic summary
start_date = prices.index.min().strftime("%Y-%m-%d")
end_date = prices.index.max().strftime("%Y-%m-%d")

best3 = perf.head(3)

lines = []
lines.append("## レポート：Momentum Strategies Across Asset Classes（ETF版）")
lines.append("")
lines.append(f"**対象期間**: {start_date} - {end_date}")
lines.append(f"**対象ETF**: {', '.join(config['tickers'])}")
lines.append(f"**リバランス**: {config['rebalance_freq']} / **コスト**: {config['transaction_cost_bps']}bps / **レバ上限**: {config['leverage_cap']}")
lines.append("")

# 1. Introduction
lines.append("### 1. Introduction to Momentum Investing")
lines.append("本分析は、PDFで示された『Absolute/Relative Momentum』や『信号分散』の枠組みに沿い、ETFユニバースで再現した。")
lines.append("モメンタムは上昇局面で有利だが転換点での損失リスクがあるため、分散とリスク配分が重要である。")
lines.append("")

# 2. Prototype Momentum Factors
lines.append("### 2. Prototype Momentum Factors")
lines.append("TSM（Absolute）とCSM（Relative）を比較した結果、Sharpe上位はTSM系が中心となった。")
lines.append("上位3戦略は以下の通り。")
lines.append("")
lines.append("```")
lines.append(best3[["Sharpe", "AnnReturn", "MaxDD", "Vol"]].to_string())
lines.append("```")
lines.append("")

# 3. Risk Management and Portfolio Construction
lines.append("### 3. Risk Management & Portfolio Construction")
lines.append("PDFのEMV/リスクパリティの考え方に基づき、選定銘柄はEqual Risk Contributionで配分した。")
lines.append("Stop-loss（-5%閾値、翌月キャッシュ）を追加し、通常戦略とStop-loss戦略の両方を評価対象に含めた。")
lines.append("")

# 4. Diversified Trend Following / Scorecard
lines.append("### 4. Diversified Trend Following / Scorecard")
lines.append("複数指標の合成（Scorecard）とシンプルDTFも試行したが、当期間ではTSM_6Mが最も高いSharpeを示した。")
lines.append("")

# 5. Seasonality
lines.append("### 5. Seasonality")
lines.append("月別平均リターンを算出し、ヒートマップで季節性を可視化した。")
lines.append("Seasonalityスコア（過去同月平均）を用いたTopKロング戦略を追加し、比較対象に含めた。")
lines.append("- 保存先: outputs/tables/seasonality_monthly_avg.csv")
lines.append("- 図: outputs/figures/seasonality_heatmap.png")
lines.append("")

# 6. Benchmark
lines.append("### 6. Benchmark (EQW)")
lines.append("EQW（月次等金額リバランス）をベンチマークとして追加し、戦略比較に含めた。")
lines.append("")

# 7. Appendices / Metrics
lines.append("### 7. Appendices (Metrics & Outputs)")
lines.append("- 詳細指標: `outputs/tables/performance_summary.csv`")
lines.append("- 戦略相関: `outputs/tables/strategy_correlation.csv`")
lines.append("- 累積パフォーマンス図: `outputs/figures/cumulative_returns.png`")
lines.append("- 相関ヒートマップ: `outputs/figures/strategy_correlation.png`")
lines.append("- Doc差分比較: outputs/figures/doc_benchmark_variants_cumret.png / doc_benchmark_variants_drawdown.png")
lines.append("- Doc差分比較(SL): outputs/figures/doc_benchmark_variants_sl_cumret.png / doc_benchmark_variants_sl_drawdown.png")
lines.append("- Doc差分比較: outputs/tables/doc_benchmark_variants_summary.csv / doc_benchmark_variants_annual_summary.csv")
lines.append("- Doc差分比較(SL): outputs/tables/doc_benchmark_variants_sl_summary.csv / doc_benchmark_variants_sl_annual_summary.csv")
lines.append("- Doc差分寄与(SL): outputs/tables/doc_benchmark_factor_contrib_sl.csv")
lines.append("- Doc差分寄与: outputs/tables/doc_benchmark_factor_contrib.csv")
lines.append("- Doc vs Scorecard: outputs/figures/doc_vs_scorecard_cumret.png / doc_vs_scorecard_drawdown.png")
if doc_largest_key is not None:
    lines.append(f"Doc差分でSharpe影響が最大の要素: {doc_largest_key} (ΔSharpe={doc_largest_val:.4f})")
lines.append("- Doc差分比較: outputs/figures/doc_benchmark_variants_cumret.png / doc_benchmark_variants_drawdown.png")
lines.append("- Doc差分比較(SL): outputs/figures/doc_benchmark_variants_sl_cumret.png / doc_benchmark_variants_sl_drawdown.png")
lines.append("- Doc差分比較: outputs/tables/doc_benchmark_variants_summary.csv / doc_benchmark_variants_annual_summary.csv")
lines.append("- Doc差分比較(SL): outputs/tables/doc_benchmark_variants_sl_summary.csv / doc_benchmark_variants_sl_annual_summary.csv")
lines.append("- Doc差分寄与(SL): outputs/tables/doc_benchmark_factor_contrib_sl.csv")

report_md = "\n".join(lines)

display(Markdown(report_md))


## 12. Validation (Synthetic Test)


### 未来情報リークの簡易チェック

以下は、バックテストが $w_{t-1}$ を用いて $t$ のリターンを計算しているか、また合成モデルの学習が当月までの情報に限定されているかを簡易的に確認するためのチェックです。


In [ ]:
# Simple future-leak checks
issues = []

# 1) Backtest alignment check: recompute returns with shift(1)
for name in returns_df.columns:
    w = all_weights.get(name)
    if w is None:
        continue
    w = w.reindex(monthly_ret.index).fillna(0.0)
    gross = (w.shift(1) * monthly_ret).sum(axis=1)
    turnover = w.diff().abs().sum(axis=1)
    cost = turnover.shift(1) * (config["transaction_cost_bps"] / 10000.0)
    net = gross - cost

    diff = (net - returns_df[name]).abs().max()
    if pd.notna(diff) and diff > 1e-10:
        issues.append((name, float(diff)))

if issues:
    print("[WARN] Backtest alignment mismatch:")
    for name, diff in issues:
        print(name, "max diff", diff)
else:
    print("[OK] Backtest alignment uses w.shift(1) (no obvious look-ahead).")

# 2) GR/BMA training index sanity: latest training target is current month
leak_flag = False
if "combo_lookback_months" in config:
    dates = monthly_ret.index
    for i, dt in enumerate(dates):
        if i < 2:
            continue
        start = max(0, i - config["combo_lookback_months"])
        idx = dates[start:i]
        if len(idx) == 0:
            continue
        latest_m = idx[-1]
        pos = dates.get_loc(latest_m)
        if pos + 1 < len(dates):
            y_date = dates[pos + 1]
            if y_date > dt:
                leak_flag = True
                break

print("[OK] GR/BMA training window uses targets up to current month only." if not leak_flag else "[WARN] Potential look-ahead in GR/BMA training window.")

# 3) Signal timing sanity
max_sig_date = max(df.index.max() for df in signals.values())
if max_sig_date > monthly_ret.index.max():
    print("[WARN] Some signals extend beyond monthly return index.")
else:
    print("[OK] Signal indices align within monthly return range.")


In [ ]:
# Simple synthetic test: constant positive returns
syn = pd.Series([0.01]*24)  # 2 years monthly
stats_syn = perf_stats(syn)
print("Synthetic test stats:")
print(stats_syn)
